In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uom190346a/sleep-health-and-lifestyle-dataset")

print("Path to dataset files:", path)

#https://www.kaggle.com/code/jillanisofttech/sleep-health-and-lifestyle-predication-with-94-ac
#eksempel av noen sin løsning på dette datasetet

Path to dataset files: C:\Users\edvar\.cache\kagglehub\datasets\uom190346a\sleep-health-and-lifestyle-dataset\versions\2


In [2]:
import pandas as pd

In [3]:
data = pd.read_csv(path+"/Sleep_health_and_lifestyle_dataset.csv")

In [4]:
headAmount = 5 # for å skrive ut alle data punkt i csv file skriv data.size() istedenfor eit tall
data.head(headAmount)

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [5]:
print(data['Sleep Disorder'].unique())
print(data['Occupation'].unique())
print(data['BMI Category'].unique())

[nan 'Sleep Apnea' 'Insomnia']
['Software Engineer' 'Doctor' 'Sales Representative' 'Teacher' 'Nurse'
 'Engineer' 'Accountant' 'Scientist' 'Lawyer' 'Salesperson' 'Manager']
['Overweight' 'Normal' 'Obese' 'Normal Weight']


In [6]:
data['Blood Pressure'].unique()

array(['126/83', '125/80', '140/90', '120/80', '132/87', '130/86',
       '117/76', '118/76', '128/85', '131/86', '128/84', '115/75',
       '135/88', '129/84', '130/85', '115/78', '119/77', '121/79',
       '125/82', '135/90', '122/80', '142/92', '140/95', '139/91',
       '118/75'], dtype=object)

In [7]:
# Vi deler opp blodtrykk siden det er to verdiar i ein kolonne
data_16_pro_xl = (pd.concat([data, data['Blood Pressure'].str.split('/', expand=True)], axis=1).
                  drop('Blood Pressure', axis=1))

In [8]:
data_16_pro_xl

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder,0,1
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,77,4200,NaN,126,83
1,2,Male,28,Doctor,6.2,6,60,8,Normal,75,10000,NaN,125,80
2,3,Male,28,Doctor,6.2,6,60,8,Normal,75,10000,NaN,125,80
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,85,3000,Sleep Apnea,140,90
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,85,3000,Sleep Apnea,140,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,Female,59,Nurse,8.1,9,75,3,Overweight,68,7000,Sleep Apnea,140,95
370,371,Female,59,Nurse,8.0,9,75,3,Overweight,68,7000,Sleep Apnea,140,95
371,372,Female,59,Nurse,8.1,9,75,3,Overweight,68,7000,Sleep Apnea,140,95
372,373,Female,59,Nurse,8.1,9,75,3,Overweight,68,7000,Sleep Apnea,140,95


In [9]:
data_16_pro_xl = data_16_pro_xl.rename(columns={0: 'Systolic Blood Pressure', 1: 'Diastolic Blood Pressure'})

In [10]:
data_16_pro_xl

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder,Systolic Blood Pressure,Diastolic Blood Pressure
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,77,4200,NaN,126,83
1,2,Male,28,Doctor,6.2,6,60,8,Normal,75,10000,NaN,125,80
2,3,Male,28,Doctor,6.2,6,60,8,Normal,75,10000,NaN,125,80
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,85,3000,Sleep Apnea,140,90
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,85,3000,Sleep Apnea,140,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,Female,59,Nurse,8.1,9,75,3,Overweight,68,7000,Sleep Apnea,140,95
370,371,Female,59,Nurse,8.0,9,75,3,Overweight,68,7000,Sleep Apnea,140,95
371,372,Female,59,Nurse,8.1,9,75,3,Overweight,68,7000,Sleep Apnea,140,95
372,373,Female,59,Nurse,8.1,9,75,3,Overweight,68,7000,Sleep Apnea,140,95


In [11]:
data_16_pro_xl['Systolic Blood Pressure'] = data_16_pro_xl['Systolic Blood Pressure'].astype(float)
data_16_pro_xl['Diastolic Blood Pressure'] = data_16_pro_xl['Diastolic Blood Pressure'].astype(float)
data_16_pro_xl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Person ID                 374 non-null    int64  
 1   Gender                    374 non-null    object 
 2   Age                       374 non-null    int64  
 3   Occupation                374 non-null    object 
 4   Sleep Duration            374 non-null    float64
 5   Quality of Sleep          374 non-null    int64  
 6   Physical Activity Level   374 non-null    int64  
 7   Stress Level              374 non-null    int64  
 8   BMI Category              374 non-null    object 
 9   Heart Rate                374 non-null    int64  
 10  Daily Steps               374 non-null    int64  
 11  Sleep Disorder            155 non-null    object 
 12  Systolic Blood Pressure   374 non-null    float64
 13  Diastolic Blood Pressure  374 non-null    float64
dtypes: float64

In [12]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
data_16_pro_xl['Occupation']= label_encoder.fit_transform(data_16_pro_xl['Occupation'])
data_16_pro_xl['BMI Category']= label_encoder.fit_transform(data_16_pro_xl['BMI Category'])
data_16_pro_xl['Sleep Disorder']= label_encoder.fit_transform(data_16_pro_xl['Sleep Disorder'])
data_16_pro_xl['Gender']= label_encoder.fit_transform(data_16_pro_xl['Gender'])

In [14]:
X = data_16_pro_xl.drop(['Sleep Disorder','Person ID'], axis=1)
y = data_16_pro_xl['Sleep Disorder']

In [15]:
import joblib
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, y_pred)}")

joblib.dump(model, 'sleep_disorder_model.pkl')

Model Accuracy: 0.9466666666666667


['sleep_disorder_model.pkl']